In [1]:
from huggingface_hub import login
login('hf_token')  # This will prompt you to enter your Hugging Face token


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",
                                            torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [3]:
import pandas as pd
classes = ["Very_Negative", "Negative", "Neutral", "Positive", "Very_Positive"]
df = pd.read_csv('/kaggle/input/sst2-data/sst2/preprocessed_data.csv')
df = df[df['labels'].isin(classes)]
df.head()

,phrase,labels
0,!,Neutral
1,! ',Neutral
2,! '',Neutral
3,! Alas,Neutral
4,! Brilliant,Very_Positive


In [4]:
df = df.sample(frac =1,random_state = 1).reset_index(drop=True).reset_index()
train_size = 0.8
train_len = int(train_size*len(df))

In [5]:
df_train = df[:train_len]
df_test = df[train_len:]

In [6]:
df_train.shape,df_test.shape

((191385, 3), (47847, 3))

In [7]:
classes = ["Very_Negative", "Negative", "Neutral", "Positive", "Very_Positive"]

# Instead of embedding \n directly inside the f-string, we'll handle it separately
classes_with_labels = "\n or ".join([f"Category: {x}" for x in classes])

system_prompt = {
    "role": "system",
    "content": f"""You are an expert in sentiment analysis who classifies the sentiment of a phrase. You have to choose only one class from the following classes:
{classes_with_labels}.
Ensure your output is from the above list only."""
}

post_message = {
    "role": "assistant",
    "content": "Category:"
}

user_messages = [
    {
        "role": "user",
        "content": f"Phrase: {row['phrase']}"
    }
    for _, row in df_test.iloc[:500].iterrows()  # Correct iteration over DataFrame rows
]

prompts = [
    [system_prompt, user_msg, post_message]
    for user_msg in user_messages
]


In [8]:
prompts[0]

[{'role': 'system',
  'content': 'You are an expert in sentiment analysis who classifies the sentiment of a phrase. You have to choose only one class from the following classes:\nCategory: Very_Negative\n or Category: Negative\n or Category: Neutral\n or Category: Positive\n or Category: Very_Positive.\nEnsure your output is from the above list only.'},
 {'role': 'user', 'content': 'Phrase: require the full emotional involvement'},
 {'role': 'assistant', 'content': 'Category:'}]

In [9]:
tokenized = tokenizer.apply_chat_template(
    prompts,
    continue_final_message = True,
    padding=True,
    return_tensors = 'pt'
)

out = model.generate(tokenized,max_new_tokens=25)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [15]:
decode = tokenizer.batch_decode(out)

In [11]:
decode[0]

'<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Nov 2024\n\nYou are an expert in sentiment analysis who classifies the sentiment of a phrase. You have to choose only one class from the following classes:\nCategory: Very_Negative\n or Category: Negative\n or Category: Neutral\n or Category: Positive\n or Category: Very_Positive.\nEnsure your output is from the above list only.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPhrase: require the full emo

In [12]:
predicted_labels = [d.split('<|start_header_id|>assistant<|end_header_id|>\n\nCategory:')[1].split('<|eot_id|>')[0].strip() for d in decode]

In [13]:
predicted_labels[:5]

['Positive\n or Category: Neutral\n or Category: Neutral\n or Category: Positive',
 'Negative',
 'Negative\n or Category: Neutral\n or Category: Neutral\n or Category: Positive\n or Category: Neutral\nThe best answer',
 'Neutral\n or Category: Negative\n or Category: Positive\n or Category: Negative',
 'Positive\nor\nor Category: Negative']

In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

true_labels = df_test['labels'][:500].tolist()  # True labels from the test dataframe

# Calculate the metrics: Accuracy, Precision, Recall, F1 Score
def calculate_metrics(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = calculate_metrics(predicted_labels, true_labels)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.1320
Precision: 0.3985
Recall: 0.1320
F1 Score: 0.1729


/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
